In [12]:
from dataloader import MinibooneLoader
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

shuffleSeed = 0
trainSize = 0.8
testSize = 0.2

In [13]:
dataset = MinibooneLoader().loadMiniboone()

events = dataset.events

In [14]:
# Set up a preprocessing pipeline
estimators = [
    # Standardizes and scales the dataset
    ['scale', StandardScaler()]
]
pipe = Pipeline(estimators).fit(events)

# Transform the data using the prepared pipeline.
stdEvents = pipe.transform(events)

In [15]:

# Shuffle the dataset and split into training and test sets.
X_train, X_test, y_train, y_test = train_test_split(stdEvents, dataset.classifications, shuffle=True, random_state=shuffleSeed, train_size=trainSize, test_size=testSize)

print('Data has been split. Training data:')
print(X_train)
print(y_train)

Data has been split. Training data:
[[ 0.04852549  0.05710245 -0.42166557 ...  0.04230077  0.08020339
   0.0614591 ]
 [ 0.08522973  0.05787857  0.40372423 ...  0.15819105  0.06338857
   0.0602966 ]
 [ 0.03886527  0.05797087 -0.56084576 ...  0.03024111  0.03142102
   0.06056402]
 ...
 [ 0.06007261  0.07552737  0.02669232 ...  0.06275209  0.08296288
   0.06197942]
 [ 0.06221417  0.06761905 -0.02986604 ...  0.07141983  0.07571434
   0.05977339]
 [ 0.06484492  0.0546634   0.00282108 ...  0.0365456   0.03887806
   0.0591355 ]]
[0 1 0 ... 1 1 1]


In [16]:
layersConfig = (10, 10, 10)
maxIterations = 1000

print('Training and fitting neural network')
mlp = MLPClassifier(hidden_layer_sizes=layersConfig, max_iter=maxIterations, verbose=True)
mlp.fit(X_train, y_train.ravel())
print('\nTraining complete\n')

print('Classifying test data')
predictions = mlp.predict(X_test)

print('\nConfusion matrix:')
print(confusion_matrix(y_test, predictions))

print('\nClassification report:')
print(classification_report(y_test, predictions))


Training and fitting neural network
Iteration 1, loss = 0.44905350
Iteration 2, loss = 0.27099337
Iteration 3, loss = 0.25248869
Iteration 4, loss = 0.24056195
Iteration 5, loss = 0.23119265
Iteration 6, loss = 0.22457859
Iteration 7, loss = 0.22012592
Iteration 8, loss = 0.21612475
Iteration 9, loss = 0.21260776
Iteration 10, loss = 0.20901799
Iteration 11, loss = 0.20569608
Iteration 12, loss = 0.20267556
Iteration 13, loss = 0.20064481
Iteration 14, loss = 0.19829653
Iteration 15, loss = 0.19595801
Iteration 16, loss = 0.19364552
Iteration 17, loss = 0.19253857
Iteration 18, loss = 0.19094063
Iteration 19, loss = 0.18983113
Iteration 20, loss = 0.18804426
Iteration 21, loss = 0.18776455
Iteration 22, loss = 0.18640499
Iteration 23, loss = 0.18525896
Iteration 24, loss = 0.18511755
Iteration 25, loss = 0.18354232
Iteration 26, loss = 0.18290478
Iteration 27, loss = 0.18271905
Iteration 28, loss = 0.18083950
Iteration 29, loss = 0.18071963
Iteration 30, loss = 0.18018836
Iteration 31,